![JohnSnowLabs](https://sparknlp.org/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp/blob/master/examples/python/llama.cpp/llama.cpp_in_Spark_NLP_AutoGGUFModel.ipynb)

# Import llama.cpp 🦙 models into Spark NLP 🚀

Let's keep in mind a few things before we start 😊

- llama.cpp support was introduced in `Spark NLP 5.5.0`, enabling quantized LLM inference on a wide range of devices. Please make sure you have upgraded to the latest Spark NLP release.
- You need to use your own `.gguf` model files, which also include the models from the [Hugging Face Models](https://huggingface.co/models?library=gguf).

## Download a GGUF Reranker Model

Lets download a GGUF model to test it out. For this, we will use [gpustack/bge-reranker-v2-m3-Q4_K_M](https://huggingface.co/gpustack/bge-reranker-v2-m3-GGUF). It is a 3.8B parameter model which also is available in 4-bit quantization.

We can download the model by selecting the q4 GGUF file from the "Files and versions" tab.

Once downloaded, we can directly import this model into Spark NLP!

In [1]:
EXPORT_PATH = "/tmp/bge-reranker-v2-m3-Q4_K_M.gguf"

In [2]:
! wget https://huggingface.co/gpustack/bge-reranker-v2-m3-GGUF/resolve/main/bge-reranker-v2-m3-Q4_K_M.gguf?download=true -O {EXPORT_PATH}

--2025-08-15 05:48:01--  https://huggingface.co/gpustack/bge-reranker-v2-m3-GGUF/resolve/main/bge-reranker-v2-m3-Q4_K_M.gguf?download=true
Resolving huggingface.co (huggingface.co)... 18.164.174.118, 18.164.174.17, 18.164.174.55, ...
Connecting to huggingface.co (huggingface.co)|18.164.174.118|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs-us-1.hf.co/repos/a8/9b/a89b7cc84f0ac1c67a4d8b3f1929339b24363e86ac58811b0ffedcfb20dfe844/e186a244ed455b4ab66ec64339ce7427a6ae13f5c0b5e544de96e50f0f8b3673?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27bge-reranker-v2-m3-Q4_K_M.gguf%3B+filename%3D%22bge-reranker-v2-m3-Q4_K_M.gguf%22%3B&Expires=1755240482&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTc1NTI0MDQ4Mn19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmhmLmNvL3JlcG9zL2E4LzliL2E4OWI3Y2M4NGYwYWMxYzY3YTRkOGIzZjE5MjkzMzliMjQzNjNlODZhYzU4ODExYjBmZmVkY2ZiMjBkZmU4NDQvZTE4NmEyNDRlZDQ1NWI0YWI2NmVj

## Import and Save AutGGUF renranker models in Spark NLP

- Let's install and setup Spark NLP (if running it Google Colab)
- This part is pretty easy via our simple script

In [2]:
# Only execute this if you are on Google Colab
! wget -q http://setup.johnsnowlabs.com/colab.sh -O - | bash

Installing PySpark 3.4.4 and Spark NLP 6.1.1
setup Colab for PySpark 3.4.4 and Spark NLP 6.1.1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.4/311.4 MB 1.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 727.0/727.0 kB 51.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
dataproc-spark-connect 0.8.3 requires pyspark[connect]~=3.5.1, but you have pyspark 3.4.4 which is incompatible.


Let's start Spark with Spark NLP included via our simple `start()` function

In [3]:
import sparknlp

# let's start Spark with Spark NLP with GPU enabled. If you don't have GPUs available remove this parameter.
spark = sparknlp.start(gpu=True)
print(sparknlp.version())

6.1.1


- Let's use the `loadSavedModel` functon in `AutoGGUFModel`
- Most params will be set automatically. They can also be set later after loading the model in `AutoGGUFModel` during runtime, so don't worry about setting them now.
- `loadSavedModel` accepts two params, first is the path to the exported model. The second is the SparkSession that is `spark` variable we previously started via `sparknlp.start()`
- NOTE: `loadSavedModel` accepts local paths in addition to distributed file systems such as `HDFS`, `S3`, `DBFS`, etc. This feature was introduced in Spark NLP 4.2.2 release. Keep in mind the best and recommended way to move/share/reuse Spark NLP models is to use `write.save` so you can use `.load()` from any file systems natively.

In [5]:
from sparknlp.annotator import *

autoGGUFModel = (
    AutoGGUFReranker.loadSavedModel(EXPORT_PATH, spark)
    .setInputCols("document")
    .setOutputCol("completions")
    .setBatchSize(4)
)

- Let's save it on disk so it is easier to be moved around and also be used later via `.load` function

In [6]:
autoGGUFModel.write().overwrite().save(f"bge_reranker_spark_nlp/")

Let's clean up stuff we don't need anymore

In [ ]:
!rm -rf {EXPORT_PATH}

Awesome  😎 !

This is your GGUF model from loaded and saved by Spark NLP 🚀

In [12]:
! ls -l bge_reranker_spark_nlp

total 428112
-rwxr-xr-x 1 root root 438376864 Aug 15 05:49 bge-reranker-v2-m3-Q4_K_M.gguf
drwxr-xr-x 2 root root      4096 Aug 15 05:49 metadata


Now let's see how we can use it on other machines, clusters, or any place you wish to use your new and shiny GGUF model 😊

In [13]:
import sparknlp
from sparknlp.base import *
from sparknlp.annotator import *
from pyspark.ml import Pipeline

document_assembler = DocumentAssembler().setInputCol("text").setOutputCol("document")

auto_gguf_model = AutoGGUFReranker.load("bge_reranker_spark_nlp").setInputCols("document").setOutputCol("reranked_documents").setQuery("A man is eating pasta.")

pipeline = Pipeline().setStages([document_assembler, auto_gguf_model])

data = (spark.createDataFrame(
                [
                    ["A man is eating food."],
                    ["A man is eating a piece of bread."],
                    ["The girl is carrying a baby."],
                    ["A man is riding a horse."],
                ]
            )
            .toDF("text")
            .repartition(1))

result = pipeline.fit(data).transform(data)
# Verify results contain relevance scores
collected_results = result.collect()
for row in collected_results:
    annotations = row["reranked_documents"]


    for annotation in annotations:
        print(annotation["result"])
        # Relevance score should be a valid number
        score = float(annotation.metadata["relevance_score"])
        print(f"Relevance score: {score}")

result.show()

A man is eating food.
Relevance score: 6.953318
A man is eating a piece of bread.
Relevance score: 1.8993418
A man is riding a horse.
Relevance score: -8.467077
The girl is carrying a baby.
Relevance score: -10.770506
+--------------------+--------------------+--------------------+
|                text|            document|  reranked_documents|
+--------------------+--------------------+--------------------+
|A man is eating f...|[{document, 0, 20...|[{document, 0, 20...|
|A man is eating a...|[{document, 0, 32...|[{document, 0, 32...|
|The girl is carry...|[{document, 0, 27...|[{document, 0, 23...|
|A man is riding a...|[{document, 0, 23...|[{document, 0, 27...|
+--------------------+--------------------+--------------------+



That's it! You can now go wild and use hundreds of GGUF models from HuggingFace 🤗 in Spark NLP 🚀
